### Importing all the necessary libraries

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, Flatten, Dense, Reshape, Lambda, Concatenate
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from skimage.metrics import structural_similarity as ssim
from PIL import Image

### Model Architecture

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Concatenate, Conv2D, MaxPooling2D, Flatten, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

# Updated Parameters
img_height, img_width, img_channels = 256, 256, 1 #Input image dimensions
latent_dim = (16, 16, 1) #Latent space dimensions
num_classes = 9 #Number of classes

class CVAE(tf.keras.Model):
    def __init__(self, latent_dim, num_classes):
        super(CVAE, self).__init__()
        self.latent_dim = latent_dim
        self.num_classes = num_classes
        self.encoder = self.build_encoder()
        self.decoder = self.build_decoder()

    def build_encoder(self):
        input_img = Input(shape=(img_height, img_width, img_channels))
        input_label = Input(shape=(self.num_classes,))
        label_embedding = Dense(img_height * img_width)(input_label)
        label_embedding = Reshape((img_height, img_width, 1))(label_embedding)

        x = Concatenate()([input_img, label_embedding])
        x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
        x = MaxPooling2D((2, 2), padding='same')(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        x = MaxPooling2D((2, 2), padding='same')(x)
        x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
        x = MaxPooling2D((2, 2), padding='same')(x)
        x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
        x = MaxPooling2D((2, 2), padding='same')(x)
        x = Flatten()(x)
        mean = Dense(self.latent_dim[0] * self.latent_dim[1] * self.latent_dim[2])(x)
        log_var = Dense(self.latent_dim[0] * self.latent_dim[1] * self.latent_dim[2])(x)
        mean = Reshape(self.latent_dim)(mean)
        log_var = Reshape(self.latent_dim)(log_var)
        return Model([input_img, input_label], [mean, log_var], name='encoder')

    def build_decoder(self):
        latent_input = Input(shape=self.latent_dim)
        input_label = Input(shape=(self.num_classes,))
        label_embedding = Dense(self.latent_dim[0] * self.latent_dim[1] * self.latent_dim[2])(input_label)
        label_embedding = Reshape(self.latent_dim)(label_embedding)

        x = Concatenate()([latent_input, label_embedding])
        x = Reshape((self.latent_dim[0] * self.latent_dim[1] * self.latent_dim[2] * 2,))(x)
        x = Dense(16 * 16 * 512, activation='relu')(x)
        x = Reshape((16, 16, 512))(x)
        x = Conv2DTranspose(256, (3, 3), activation='relu', strides=2, padding='same')(x)
        x = Conv2DTranspose(128, (3, 3), activation='relu', strides=2, padding='same')(x)
        x = Conv2DTranspose(64, (3, 3), activation='relu', strides=2, padding='same')(x)
        x = Conv2DTranspose(32, (3, 3), activation='relu', strides=2, padding='same')(x)
        output_img = Conv2D(img_channels, (3, 3), activation='sigmoid', padding='same')(x)
        return Model([latent_input, input_label], output_img, name='decoder')

    def sampling(self, args):
        mean, log_var = args
        epsilon = tf.random.normal(shape=tf.shape(mean), mean=0., stddev=1.)
        return mean + tf.exp(log_var / 2) * epsilon

    def call(self, inputs):
        input_img, input_label = inputs
        mean, log_var = self.encoder([input_img, input_label])
        z = self.sampling([mean, log_var])
        reconstructed = self.decoder([z, input_label])
        reconstruction_loss = tf.reduce_mean(MeanSquaredError()(input_img, reconstructed))
        kl_loss = -0.5 * tf.reduce_mean(1 + log_var - tf.square(mean) - tf.exp(log_var))
        self.add_loss(reconstruction_loss + kl_loss)
        return reconstructed

cvae = CVAE(latent_dim, num_classes)
cvae.compile(optimizer=Adam())

cvae.encoder.summary()
cvae.decoder.summary()

### Loading the saved model weights

In [ ]:
# Building the model by running a dummy input through it
dummy_image = tf.random.normal((1, 256, 256, 1))  # Dummy input for the image
dummy_composition = tf.random.normal((1, 9))  # Dummy input for the composition
_ = cvae([dummy_image, dummy_composition])  # Passing the dummy inputs to create variables

# Loading the saved weights
cvae.load_weights('saved_model_weights.h5')

print("Model weights reloaded successfully.")

### Performing Cubic interpolation to generate images of desired composition

In [ ]:
from scipy.interpolate import CubicSpline
from utils import extract_c_avg, calculate_rounded_avg

num_interpolations = 50
target_rounded_avg = extract_c_avg('inter_comp.dat') # Extracting targeted composition value from inter_comp.dat file
tolerance = 0.001
desired_num_images = 500 # Number of images of target_rounded_avg to generate
output_folder = f'{target_rounded_avg}_gen'
label_file = f'interpolated_labels_{target_rounded_avg}.txt'
os.makedirs(output_folder, exist_ok=True)
gen_folder = output_folder

def generate_images_for_rounded_avg(cvae, start_label, intermediate_label_2, intermediate_label_3, intermediate_label_4, intermediate_label_5, intermediate_label_6, intermediate_label_7, intermediate_label_8, end_label, target_rounded_avg, tolerance, desired_num_images):
    generated_images_info = []

    start_label = np.array(start_label)
    intermediate_label_2 = np.array(intermediate_label_2)
    intermediate_label_3 = np.array(intermediate_label_3)
    intermediate_label_4 = np.array(intermediate_label_4)
    intermediate_label_5 = np.array(intermediate_label_5)
    intermediate_label_6 = np.array(intermediate_label_6)
    intermediate_label_7 = np.array(intermediate_label_7)
    intermediate_label_8 = np.array(intermediate_label_8)
    end_label = np.array(end_label)

    x_vals = [0.27, 0.31, 0.35, 0.39, 0.40, 0.42, 0.44, 0.46, 0.48]  # The positions of start and end points
    y_vals = np.vstack([start_label, intermediate_label_2, intermediate_label_3, intermediate_label_4, intermediate_label_5, intermediate_label_6, intermediate_label_7, intermediate_label_8, end_label])
    cubic_interp_func = CubicSpline(x_vals, y_vals, axis=0)
    
    if target_rounded_avg >= 0.27 and target_rounded_avg < 0.29:
        start = 0.27
        end = 0.29
    elif target_rounded_avg >=0.29 and target_rounded_avg < 0.31:
        start = 0.29
        end = 0.31
    elif target_rounded_avg == 0.31 :
        start = 0.31
        end = 0.33
    elif target_rounded_avg > 0.31 and target_rounded_avg <=0.33:
        start = 0.31
        end = 0.33
    elif target_rounded_avg >0.33 and target_rounded_avg < 0.35:
        start = 0.33
        end = 0.35
    elif target_rounded_avg == 0.35 :
        start = 0.35
        end = 0.37
    elif target_rounded_avg > 0.35 and target_rounded_avg <=0.37:       
        start = 0.35
        end = 0.37
    elif target_rounded_avg >0.37 and target_rounded_avg < 0.39:
        start = 0.37
        end = 0.39
    elif target_rounded_avg == 0.39 :
        start = 0.39
        end = 0.40
    elif target_rounded_avg >0.39 and target_rounded_avg <0.40:
        start = 0.39
        end = 0.40
    elif target_rounded_avg == 0.40:
        start = 0.40
        end = 0.41
    elif target_rounded_avg == 0.41:
        start = 0.40
        end = 0.41
    elif target_rounded_avg == 0.42:
        start = 0.42
        end = 0.43
    elif target_rounded_avg == 0.43:
        start = 0.42
        end = 0.43
    elif target_rounded_avg == 0.44:
        start = 0.44
        end = 0.45
    elif target_rounded_avg == 0.45:
        start = 0.44
        end = 0.45
    elif target_rounded_avg == 0.46:
        start = 0.46
        end = 0.47
    elif target_rounded_avg == 0.47:
        start = 0.47
        end = 0.48
    elif target_rounded_avg == 0.48:
        start = 0.47
        end = 0.48
    
    with open(label_file, 'w') as label_output:
        while len(generated_images_info) < desired_num_images:
            for alpha in np.linspace(start, end, num_interpolations):
                if len(generated_images_info) >= desired_num_images:
                    break

                interpolated_label = cubic_interp_func(alpha)
                random_latent_vector = tf.random.normal(shape=(1, *latent_dim))
                interpolated_label_tensor = tf.convert_to_tensor(interpolated_label.reshape(1, -1), dtype=tf.float32)
                generated_image = cvae.decoder([random_latent_vector, interpolated_label_tensor]).numpy().reshape(img_height, img_width)
                generated_image_uint8 = (generated_image * 255).astype(np.uint8)
                rounded_avg = calculate_rounded_avg(generated_image_uint8)

                if abs(rounded_avg - target_rounded_avg) <= tolerance:
                    generated_images_info.append((generated_image_uint8, rounded_avg))
                    label_output.write(f"image_{len(generated_images_info):03d}_interpolated_label_{interpolated_label}\n")

    return generated_images_info

start_label = [1, 0, 0, 0, 0, 0, 0, 0, 0]  # Start label
intermediate_label_2 = [0, 1, 0, 0, 0, 0, 0, 0, 0]  # Intermediate label_2
intermediate_label_3 = [0, 0, 1, 0, 0, 0, 0, 0, 0]  # Intermediate label_3
intermediate_label_4 = [0, 0, 0, 1, 0, 0, 0, 0, 0]  # Intermediate label_4
intermediate_label_5 = [0, 0, 0, 0, 1, 0, 0, 0, 0]  # Intermediate label_5
intermediate_label_6 = [0, 0, 0, 0, 0, 1, 0, 0, 0]  # Intermediate label_6
intermediate_label_7 = [0, 0, 0, 0, 0, 0, 1, 0, 0]  # Intermediate label_7
intermediate_label_8 = [0, 0, 0, 0, 0, 0, 0, 1, 0]  # Intermediate label_8

end_label = [0, 0, 0, 0, 0, 0, 0, 0, 1]  # End label

generated_images_info = generate_images_for_rounded_avg(cvae, start_label, intermediate_label_2, intermediate_label_3, intermediate_label_4, intermediate_label_5, intermediate_label_6, intermediate_label_7, intermediate_label_8, end_label, target_rounded_avg, tolerance, desired_num_images)

for i, (image, rounded_avg) in enumerate(generated_images_info):
    filename = f"image_{i+1:03d}_rounded_avg_{rounded_avg:.2f}.png"
    filepath = os.path.join(output_folder, filename)
    cv2.imwrite(filepath, image)

print(f"Generated and saved {len(generated_images_info)} images with target rounded average {target_rounded_avg}.")

#### Ensuring that each interpolated lable lies in a single line in the .txt file

In [ ]:
from utils import fix_file_format_and_replace

input_file = label_file  
fix_file_format_and_replace(input_file)

### Obtaining filename and average area of white shapes with the largest, smallest and middle average area

In [ ]:
folder_path = gen_folder

image_info = []

for filename in os.listdir(folder_path):
    if filename.endswith(".png"):
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        _, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        total_area = sum(cv2.contourArea(contour) for contour in contours)
        
        total_shapes = len(contours)
        
        if total_shapes>0:
            avg_area = total_area/total_shapes
        else:
            avg_area = 0

        image_info.append((filename, avg_area))

if image_info:
    # Sort images by average area
    image_info.sort(key=lambda x: x[1])

    max_filename, max_area = image_info[-1]
    min_filename, min_area = image_info[0]
    mid_index = len(image_info) // 2
    mid_filename, mid_area = image_info[mid_index]
    
    middle_image_name = mid_filename
    middle_image_area = mid_area
    
    if target_rounded_avg < 0.29:
        initial_image_name = min_filename
        initial_image_area = min_area
        
    print(f"Smallest_avg_area: {min_filename} - Average_area: {min_area}")
    print(f"Middle_avg_area: {mid_filename} - Average_area: {mid_area}")
    print(f"Largest_avg_area: {max_filename} - Average_area: {max_area}")
else:
    print("No images found in the specified folder.")

### Performing Similarity Search to find the image corresponding to initial timestep value 

In [ ]:
from utils import compute_ssim_filtered_image_info

if target_rounded_avg >=0.27 and target_rounded_avg <0.29:
    data = np.genfromtxt(f'dataset/comp{int((0.27)*100)}_350.dat', dtype=float)
    
if target_rounded_avg >=0.29 and target_rounded_avg <=0.31:
    data = np.genfromtxt(f'datset/comp{int((0.31)*100)}_301.dat', dtype=float)
    
elif target_rounded_avg >0.31 and target_rounded_avg <=0.33:
    data = np.genfromtxt(f'dataset/comp{int((0.31)*100)}_301.dat', dtype=float)
        
elif target_rounded_avg >0.33 and target_rounded_avg <=0.35:
    data = np.genfromtxt(f'dataset/comp{int((0.35)*100)}_301.dat', dtype=float)
    
elif target_rounded_avg >0.35 and target_rounded_avg <=0.37:
    data = np.genfromtxt(f'dataset/comp{int((0.35)*100)}_301.dat', dtype=float)
    
elif target_rounded_avg >0.37 and target_rounded_avg <=0.39:
    data = np.genfromtxt(f'dataset/comp{int((0.39)*100)}_301.dat', dtype=float)

elif target_rounded_avg == 0.40:
    data = np.genfromtxt(f'dataset/comp{int((0.40)*100)}_301.dat', dtype=float)
    
elif target_rounded_avg == 0.41:
    data = np.genfromtxt(f'dataset/comp{int((0.40)*100)}_301.dat', dtype=float)
    
elif target_rounded_avg ==0.42:
    data = np.genfromtxt(f'dataset/comp{int((0.42)*100)}_301.dat', dtype=float)
        
elif target_rounded_avg ==0.43:
    data = np.genfromtxt(f'dataset/comp{int((0.42)*100)}_301.dat', dtype=float)
    
elif target_rounded_avg == 0.44:
    data = np.genfromtxt(f'dataset/comp{int((0.44)*100)}_301.dat', dtype=float)
    
elif target_rounded_avg ==0.45:
    data = np.genfromtxt(f'dataset/comp{int((0.44)*100)}_301.dat', dtype=float)
    
elif target_rounded_avg ==0.46:
    data = np.genfromtxt(f'dataset/comp{int((0.46)*100)}_301.dat', dtype=float)
    
elif target_rounded_avg ==0.47:
    data = np.genfromtxt(f'dataset/comp{int((0.48)*100)}_375.dat', dtype=float)
    
elif target_rounded_avg ==0.48:
    data = np.genfromtxt(f'dataset/comp{int((0.48)*100)}_301.dat', dtype=float)
    

avg = np.mean(data)
data_normalized = 255 * (data - np.min(data)) / (np.max(data) - np.min(data))
data_uint8 = data_normalized.astype(np.uint8)
reference_image = Image.fromarray(data_uint8, mode='L')
reference_image.save(f'initial_reference_image_for_{target_rounded_avg}.png')

reference_image = cv2.imread(f'initial_reference_image_for_{target_rounded_avg}.png', cv2.IMREAD_GRAYSCALE)

folder_path = gen_folder

image_info = compute_ssim_filtered_image_info(reference_image, gen_folder, threshold=70)

if image_info:
    min_filename, min_area = image_info[0]
    initial_image_name = min_filename
    initial_image_area = min_area
    print(f"Smallest_avg_area: {min_filename} - Average_area: {min_area}")
else:
    print("No images with similarity ≥ 60% found in the specified folder.")

### Performing Similarity Search to find the image corresponding to final timestep value 

In [ ]:
from utils import compute_ssim_filtered_image_info
    
if target_rounded_avg >=0.27 and target_rounded_avg <0.29:
    data = np.genfromtxt(f'dataset/comp{int((0.27)*100)}_1000.dat', dtype=float)
    
elif target_rounded_avg >=0.29 and target_rounded_avg <=0.31:
    data = np.genfromtxt(f'dataset/comp{int((0.31)*100)}_1000.dat', dtype=float)
    
elif target_rounded_avg >0.31 and target_rounded_avg <=0.33:
    data = np.genfromtxt(f'dataset/comp{int((0.31)*100)}_1000.dat', dtype=float)
        
elif target_rounded_avg >0.33 and target_rounded_avg <=0.35:
    data = np.genfromtxt(f'dataset/comp{int((0.35)*100)}_1000.dat', dtype=float)
    
elif target_rounded_avg >0.35 and target_rounded_avg <=0.37:
    data = np.genfromtxt(f'dataset/comp{int((0.35)*100)}_1000.dat', dtype=float)
    
elif target_rounded_avg >0.37 and target_rounded_avg <=0.39:
    data = np.genfromtxt(f'dataset/comp{int((0.39)*100)}_1000.dat', dtype=float)
    
elif target_rounded_avg == 0.40:
    data = np.genfromtxt(f'dataset/comp{int((0.40)*100)}_1000.dat', dtype=float)
    
elif target_rounded_avg == 0.41:
    data = np.genfromtxt(f'dataset/comp{int((0.40)*100)}_1000.dat', dtype=float)
    
elif target_rounded_avg ==0.42:
    data = np.genfromtxt(f'dataset/comp{int((0.42)*100)}_1000.dat', dtype=float)
        
elif target_rounded_avg ==0.43:
    data = np.genfromtxt(f'dataset/comp{int((0.42)*100)}_1000.dat', dtype=float)
    
elif target_rounded_avg == 0.44:
    data = np.genfromtxt(f'dataset/comp{int((0.44)*100)}_1000.dat', dtype=float)
    
elif target_rounded_avg ==0.45:
    data = np.genfromtxt(f'dataset/comp{int((0.44)*100)}_1000.dat', dtype=float)
    
elif target_rounded_avg ==0.46:
    data = np.genfromtxt(f'dataset/comp{int((0.46)*100)}_1000.dat', dtype=float)
    
elif target_rounded_avg ==0.47:
    data = np.genfromtxt(f'dataset/comp{int((0.48)*100)}_1000.dat', dtype=float)
    
elif target_rounded_avg ==0.48:
    data = np.genfromtxt(f'dataset/comp{int((0.48)*100)}_1000.dat', dtype=float)

avg = np.mean(data)
data_normalized = 255 * (data - np.min(data)) / (np.max(data) - np.min(data))
data_uint8 = data_normalized.astype(np.uint8)
reference_image = Image.fromarray(data_uint8, mode='L')
reference_image.save(f'final_reference_image_for_{target_rounded_avg}.png')

reference_image = cv2.imread(f'final_reference_image_for_{target_rounded_avg}.png', cv2.IMREAD_GRAYSCALE)

folder_path = gen_folder

# Compare with generated images
image_info = compute_ssim_filtered_image_info(reference_image, gen_folder, threshold=60)

# Extract image with max average area
if image_info:
    max_filename, max_area = image_info[-1]
    final_image_name = max_filename
    final_image_area = max_area
    print(f"Largest_avg_area: {max_filename} - Average_area: {max_area}")
else:
    print("No images with similarity ≥ 60% found in the specified folder.")

### Obtaining the filename and average area of the image with the middle timestep value

In [ ]:
from utils import calculate_average_area

for filename in os.listdir(gen_folder):
    if filename.endswith(".png"):
        image_path = os.path.join(gen_folder, filename)
        avg_area = calculate_average_area(image_path)
        if(abs(avg_area - ((initial_image_area + final_image_area)/2))< 1):
            print(f"expected middle average area:{((initial_image_area + final_image_area)/2)}")
            print(f"Middle Image_name: {filename}, Middle Average Area: {avg_area:.2f}")
            middle_image_name = filename
            middle_image_area = avg_area
            break

### Obtaining the interpolated labels corresponding to the intial, middle and final timestep images

In [ ]:
from utils import get_interpolated_label

# Example usage
image_filename_1 = initial_image_name
image_filename_2 = middle_image_name
image_filename_3 = final_image_name
text_file_path = label_file  # Replacing with the actual path to your .txt file
initial_interpolated_label = get_interpolated_label(image_filename_1, text_file_path)
middle_interpolated_label = get_interpolated_label(image_filename_2, text_file_path)
final_interpolated_label = get_interpolated_label(image_filename_3, text_file_path)
print(f"initial_label: {initial_interpolated_label}")
print(f"middle_label: {middle_interpolated_label}")
print(f"final_label: {final_interpolated_label}")

### Performing SLERP (Spherical Linear Interpolation) to obtain the consistent evolution between intial and middle timestep image

### SLERP_1

In [ ]:
from utils import generate_interpolated_images_slerp1

image_folder = gen_folder # Path to folder containing generated images
img1_path = os.path.join(image_folder, initial_image_name)
img2_path = os.path.join(image_folder, middle_image_name)

n_images = 500 # Number of images to generate

slerp_middle_label, slerp_image_area = generate_interpolated_images_slerp1(cvae, img1_path, img2_path, n_images,
                                                                           initial_image_area = initial_image_area, 
                                                                           middle_image_area = middle_image_area, target_rounded_avg=target_rounded_avg, 
                                                                           initial_interpolated_label=initial_interpolated_label,
                                                                           middle_interpolated_label=middle_interpolated_label)

### Extracting the interpolated label of the last generated image by SLERP in appropriate format 

In [ ]:
# Cleaning the slerp_middle_label
cleaned_label = slerp_middle_label.strip("[]")
slerp_middle_interp_label =  ", ".join(cleaned_label.split())

print(slerp_middle_interp_label)
print(slerp_image_area)

### Performing SLERP (Spherical Linear Interpolation) to obtain the consistent evolution between middle and final timestep image

### SLERP_2

In [ ]:
from utils import generate_interpolated_images_slerp2

middle_folder = f'{target_rounded_avg}_slerp_2_1'
image_folder = gen_folder  # Path to folder with generated images

# Set img1_path and img2_path
img1_path = os.path.join(middle_folder, f"generated_slerp_1_image_{n_images}.png")

img2_path = os.path.join(image_folder, final_image_name)

generate_interpolated_images_slerp2(cvae, img1_path, img2_path, n_images, initial_image_area = slerp_image_area, final_image_area=final_image_area, 
                                    target_rounded_avg=target_rounded_avg, slerp_middle_interp_label = slerp_middle_interp_label, 
                                    final_interpolated_label= final_interpolated_label)

### Moving images generated by SLERP_1 and SLERP_2 into a single folder

In [ ]:
from utils import move_images_to_overall

folder_1 = f'{target_rounded_avg}_slerp_2_1'
folder_2 = f'{target_rounded_avg}_slerp_2_1_2'
overall_folder = f'{target_rounded_avg}_slerp_overall'

move_images_to_overall(folder_1, folder_2, overall_folder)